In [12]:
import fitz
fileName = "julio 2025"
year = "2025"
op = "AccountNu"
pdfFileName = rf"Data\pdfFiles\{op}\{fileName}.pdf"
htmlFileName = rf"Data\htmlFiles\{op}\{fileName}.html"
doc = fitz.open(pdfFileName)
html = ''

for num_pagina in range(1,len(doc)):
    pagina = doc.load_page(num_pagina)
    html += pagina.get_text("html")

with open(htmlFileName, 'w', encoding="utf-8") as f:
    f.write(html)


In [26]:
def to_num(x: str):
    return float(x.replace('$','').replace(',','').replace(' ',''))

def is_date(x:str):
    temp = x.split(' ')
    months = set([
        "ENE", "FEB", "MAR", "ABR", "MAY", "JUN",
        "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"
    ])
    try:
        int(x[0])
    except:
        return False
    if(len(temp) < 2): return False

    return (temp[1] in months)

def to_date(x:str):
    temp = x.split(' ')
    months = {
        "ENE": "01",
        "FEB": "02",
        "MAR": "03",
        "ABR": "04",
        "MAY": "05",
        "JUN": "06",
        "JUL": "07",
        "AGO": "08",
        "SEP": "09",
        "OCT": "10",
        "NOV": "11",
        "DIC": "12"
    }
    temp[1]=months[temp[1]]
    temp.append(year)

    return temp[0]+'-'+temp[1]+'-'+temp[2]

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "lxml")
tags = soup.find_all("span")
headers = ["timestamp", "type", "description", "value"]
data = []
i = 0

while i < len(tags):
    tag = tags[i].text.strip()
    while(not is_date(tag) and tag != "Saldo final del periodo"):
        i+=1
        tag = tags[i].text.strip()
    if(tag == "Saldo final del periodo"): break
    else:
        temp = []
        for j in range(4):
            if(j == 0): temp.append(to_date(tag))
            elif(j == 3): temp.append(to_num(tag))
            else: temp.append(tag)
            i+=1
            tag = tags[i].text.strip()
        data.append(temp)



In [29]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "lxml")
tags = soup.find_all("span")
headers = ["timestamp", "description", "value"]
data = []
i = 0
last = "Con estos movimientos, tu saldo promedio del periodo fue de"
for i in range(len(tags)): print(tags[i].text.strip())

while i < len(tags):
    tag = tags[i].text.strip()
    while(not is_date(tag) and not last in tag):
        i+=1
        tag = tags[i].text.strip()
    if(last in tag): break
    temp = []
    for j in range(3):
        print(tag)
        if(j == 0): temp.append(to_date(tag))
        elif(j == 2): temp.append(to_num(tag))
        else: temp.append(tag)
        i+=1
        tag = tags[i].text.strip()
    i+=1

print(data)


Detalle de movimientos en tu cuenta
FECHA
DEL 01 AL 31 JUL 2025 (31 DÍAS)                                      MONTO EN PESOS MEXICANOS
31 JUL 2025
Retiro de Cajita: Cajita Turbo
+$150.00
31 JUL 2025
Irving Ignacio Belmontes Diaz Transferencia
-$240.00
Transferencia SPEI, Hora: 18:26:41, Enviado a STP. Al cliente Irving
Ignacio Belmontes Diaz (Dato no verificado por esta institución), por
concepto Transferencia. A la cuenta 646013206886722456 clabe,
Clave de rastreo NU38HG4RK1O29VQRQNSUMSK2MH27, Clave de
referencia 310725
31 JUL 2025
Retiro de Cajita: Cajita Turbo
+$240.00
31 JUL 2025
Jonathan Raymundo Torres Cardenas  Transferencia
-$85.00
Transferencia SPEI, Hora: 13:25:18, Enviado a BBVA MEXICO. Al
cliente Jonathan Raymundo Torres Cardenas  (Dato no verificado por
esta institución), por concepto Transferencia. A la cuenta
4152314125427693 debit-card, Clave de rastreo
NU38HF1H8B6T973O2KRT3GMNQLEF, Clave de referencia 310725
31 JUL 2025
Retiro de Cajita: Cajita Turbo
+$85.00
30 JUL 20

IndexError: list index out of range

In [ ]:
import csv

with open(rf"Data\parsed\{fileName}.csv", 'w', newline='', encoding="utf-8") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(headers)
    writer.writerows(data)
